## This code is the study about feature importance of ESALQ experiment

In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller
from numpy import log
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import acf
from datetime import datetime, timedelta
from pycaret.regression import *

In [2]:
#função para tratar campo data
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df = pd.read_excel('biometria.xlsx', sheet_name='Página1')
print(df.tail(10))

     Index    peso  comprimento_Total  comprimento_Padrao  comprimento_Cabeca  \
664  665.0  492.04               29.0                24.8                 8.0   
665  666.0  454.02               27.0                23.0                 7.6   
666  667.0  480.56               27.1                22.8                 7.5   
667  668.0  412.75               25.7                22.0                 7.5   
668  669.0  471.80               27.2                23.4                 7.4   
669  670.0  382.79               25.4                21.5                 7.5   
670  671.0  401.90               26.5                22.3                 7.2   
671  672.0  441.98               27.8                23.4                 7.5   
672  673.0  468.54               28.0                23.5                 7.8   
673  674.0  418.14               26.3                22.2                 7.4   

     altura  espessura       data  
664     9.1        3.2 2024-06-12  
665     9.0        3.2 2024-06-12  


In [3]:
#df.nunique(axis='columns')
df['data'].unique()

array(['2024-02-28T00:00:00.000000000', '2024-03-20T00:00:00.000000000',
       '2024-04-10T00:00:00.000000000', '2024-05-02T00:00:00.000000000',
       '2024-05-23T00:00:00.000000000', '2024-06-12T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [4]:
df1=df[df['data']=='2024-02-28T00:00:00.000000000']
df2=df[df['data']=='2024-03-20T00:00:00.000000000']
df3=df[df['data']=='2024-04-10T00:00:00.000000000']
df4=df[df['data']=='2024-05-02T00:00:00.000000000']
df5=df[df['data']=='2024-05-23T00:00:00.000000000']
df6=df[df['data']=='2024-06-12T00:00:00.000000000']

In [5]:
def drop_columns(df):
    """
    Drops the specified columns from the DataFrame and returns the resulting DataFrame.
    Also prints the first few rows of the resulting DataFrame.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame.
    
    Returns:
    pd.DataFrame: The DataFrame after dropping the specified columns.
    """
    columns_to_drop = ['data', 'Index', 'espessura']
    df = df.drop(columns=columns_to_drop)
    return df

In [6]:
# List of DataFrames
dataframes = [df1, df2, df3, df4, df5, df6]

# Apply drop_columns function to each DataFrame in the list
dropped_dataframes = [drop_columns(df) for df in dataframes]

# Print the first few rows of each resulting DataFrame
for i, df in enumerate(dropped_dataframes):
    print(f"DataFrame {i+1} after dropping columns:")
    print(df.head())

DataFrame 1 after dropping columns:
    peso  comprimento_Total  comprimento_Padrao  comprimento_Cabeca  altura
0  51.45               14.3                12.2                 4.0     4.1
1  37.15               12.4                10.9                 3.8     3.2
2  47.26               13.7                11.2                 3.4     4.0
3  36.67               12.9                11.1                 3.7     3.7
4  45.48               13.2                11.3                 3.8     4.5
DataFrame 2 after dropping columns:
       peso  comprimento_Total  comprimento_Padrao  comprimento_Cabeca  altura
120  100.40              16.08               13.08                 4.4     5.4
121   77.45              15.70               13.50                 4.2     5.1
122   89.55              15.70               13.40                 4.2     5.1
123   88.03              16.00               13.80                 4.1     5.3
124   96.96              16.40               14.00                 4.3     5.

In [7]:
#### DATA for BLIND TEST
#exp2= setup(data = data,  target = 'peso')

#exp1= setup(data = df6,  target = 'peso')
#best_model = exp1.compare_models(n_select=1)

In [8]:
#print(best_model)

In [9]:
#huber = exp1.create_model('huber')

In [10]:
# dashboard of pycaret
#evaluate_model(huber)

In [11]:
#plot_model(huber, plot = 'feature')

In [12]:
# dashboard of pycaret
#evaluate_model(rf)

In [13]:
def automate_modeling(dataframes, target, models_dict):
    results = {}
    for i, df in enumerate(dataframes):
        print(f"Processing DataFrame {i+1}...")
        exp = setup(data=df, target=target, verbose=False)
        best_model = compare_models(n_select=1)
        bm_abbreviation = [abbr for name, abbr in models_dict.items() if best_model.__class__.__name__ in name]
        
        if bm_abbreviation:
            bm = create_model(bm_abbreviation[0])
            plot_model(bm, plot='feature')
            results[f'df{i+1}'] = bm
        else:
            print(f"No matching abbreviation found for the best model: {best_model.__class__.__name__}")

    return results

# Example usage
dataframes = dropped_dataframes
target_column = 'peso'
models_dict = {
    'LinearRegression': 'lr',
    'LassoRegression': 'lasso',
    'RidgeRegression': 'ridge',
    'ElasticNet': 'en',
    'LeastAngleRegression': 'lar',
    'LassoLeastAngleRegression': 'llar',
    'OrthogonalMatchingPursuit': 'omp',
    'BayesianRidge': 'br',
    'AutomaticRelevanceDetermination': 'ard',
    'PassiveAggressiveRegressor': 'par',
    'RandomSampleConsensus': 'ransac',
    'TheilSenRegressor': 'tr',
    'HuberRegressor': 'huber',
    'KernelRidge': 'kr',
    'SupportVectorMachine': 'svm',
    'KNeighborsRegressor': 'knn',
    'DecisionTree': 'dt',
    'RandomForest': 'rf',
    'ExtraTreesRegressor': 'et',
    'AdaBoostRegressor': 'ada',
    'GradientBoosting': 'gbr',
    'MultiLevelPerceptron': 'mlp',
    'ExtremeGradientBoosting': 'xgboost',
    'LightGradientBoosting': 'lightgbm',
    'CatBoostRegressor': 'catboost'
}

# Call the function
results = automate_modeling(dataframes, target_column, models_dict)

Processing DataFrame 1...


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:26:06
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Linear Regression


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)


Processing:   0%|          | 0/81 [00:00<?, ?it/s]